In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [7]:
# Caminho do modelo treinado
model_path = os.path.join("model", "modelo_solda.h5")
print(os.path.exists(model_path))

True


In [9]:
import h5py
with h5py.File(model_path, "r") as f:
    print(list(f.keys()))  # Deve exibir informações do modelo


OSError: Unable to synchronously open file (file signature not found)

In [8]:
# Verifica se o modelo existe antes de carregar
if os.path.exists(model_path):
    model = tf.keras.models.load_model(model_path)
    print(f"Modelo carregado: {model_path}")
else:
    raise FileNotFoundError(f"Modelo não encontrado em {model_path}")

# Diretórios de entrada e referência
input_dir = "novas_imagens"  # Criar se ainda não existir
train_dir = os.path.join("imagen", "train")
classes = ["boa", "ruim"]

OSError: Unable to synchronously open file (file signature not found)

In [ ]:
# Definir o diretório de dados
data_dir = "image"  # Substitua pelo caminho correto para seus dados

# Criar listas para armazenar imagens e rótulos
images = []
labels = []
img_size = 128  # Tamanho para redimensionamento das imagens

# Carregar e processar imagens
for class_idx, class_name in enumerate(classes):
    class_path = os.path.join(data_dir, class_name)
    if not os.path.exists(class_path):
        raise FileNotFoundError(f"Diretório não encontrado: {class_path}")
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Erro ao carregar imagem: {img_path}")
            continue
        img = cv2.resize(img, (img_size, img_size))
        images.append(img)
        labels.append(class_idx)

In [ ]:
# Converter listas para arrays numpy
images = np.array(images) / 255.0  # Normalizar os pixels (0-1)
labels = np.array(labels)

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Criar modelo CNN
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(classes), activation='softmax')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Treinar o modelo
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
# Salvar modelo para usar na classificação
model.save(r"model/modelo_solda.h5")
print("Modelo treinado e salvo com sucesso!")